# Домашнее задание № 7

## Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете

In [24]:
from string import punctuation
from nltk.corpus import wordnet as wn
from tqdm.auto import tqdm

In [25]:
def normalize(sentence: str) -> list:
    return [word.lower() for word in sentence.split() if word not in punctuation]

In [26]:
def lesk(word: str, sentence: str) -> int:
    bestsense = 0
    maxoverlap = 0
    normalized_sent = normalize(sentence)
    for i, syns in enumerate(wn.synsets(word)):
        definition = normalize(syns.definition())
        overlaps = sum(1 for word in definition if word in normalized_sent)
        if overlaps > maxoverlap:
            maxoverlap = overlaps
            bestsense = i
    return bestsense

In [27]:
def run_experiment():
    corpus_wsd = []
    corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
    for sent in corpus:
        corpus_wsd.append([s.split('\t') for s in sent.split('\n')])
    results = []
    for item in tqdm(corpus_wsd):
        try:
            orig_sentence = ' '.join([i[2] for i in item])
        except IndexError:
            continue
        for token in [i for i in item if i[0]]:
            target_word = token[1]
            ground_truth = wn.lemma_from_key(token[0]).synset()
            lesk_prediction = wn.synsets(target_word)[lesk(target_word, orig_sentence)]
            if ground_truth == lesk_prediction:
                results.append(True)
            else:
                results.append(False)
    print(f'Accuracy = {sum(1 for i in results if i) / len(results)}')

In [28]:
run_experiment()

  0%|          | 0/49453 [00:00<?, ?it/s]

Accuracy = 0.3239424291305598


Результат не очень хороший. Можно попробовать внести коррективы: исключать из списка сравнений стоп-слова.

In [29]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def normalize(sentence: str) -> list:
    return [word.lower() for word in sentence.split() if word not in punctuation and word not in stop_words]

In [30]:
run_experiment()

  0%|          | 0/49453 [00:00<?, ?it/s]

Accuracy = 0.4042298666599976


Результат несколько улучшился. Теперь попробуем при сравнении учитывать только уникальные слова без учета повторений.

In [31]:
def normalize(sentence: str) -> list:
    return {word.lower() for word in sentence.split() if word not in punctuation and word not in stop_words}

In [32]:
run_experiment()

  0%|          | 0/49453 [00:00<?, ?it/s]

Accuracy = 0.40645984169261357


Улучшение незначительно, но тем не менее, это лучший показатель accuracy среди всех вариантов.